In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

from torch.utils.data import DataLoader

from src.dataset.dataset import FurnitureStateDataset

from src.dataset.dataloader import EndlessDataloader

import numpy as np

from src.visualization.render_mp4 import create_mp4_jupyter

In [ ]:
def get_demo_data_loader(control_mode, batch_size, num_workers=4) -> DataLoader:
    demo_data = FurnitureStateDataset(
        dataset_paths=Path(
            "/data/scratch/ankile/furniture-data/processed/sim/one_leg/teleop/low/success.zarr"
        ),
        pred_horizon=1,
        obs_horizon=1,
        action_horizon=1,
        normalizer=None,
        data_subset=None,
        control_mode=control_mode,
        first_action_idx=0,
        pad_after=False,
        max_episode_count=None,
    )

    demo_data_loader = EndlessDataloader(
        dataset=demo_data,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True,
        pin_memory=True,
        drop_last=False,
        persistent_workers=False,
    )

    return demo_data_loader


In [ ]:
dataset = get_demo_data_loader("delta", 32).dataset

In [ ]:
# Get the episode data
ep_ends = dataset.episode_ends

images_list = np.split(dataset.train_data["color_image2"], ep_ends[:-1])
images_list = [image.numpy() for image in images_list]

In [ ]:
for i, images in enumerate(images_list[:]):
    create_mp4_jupyter(images, f"demo_{i}.mp4", fps=20)